In [53]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

In [86]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver

from dotenv import load_dotenv

load_dotenv()

True

In [87]:
graph = Neo4jGraph()

In [100]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI

# Step 1: Load Document
loader = TextLoader(file_path="input/hetangyuese_zhuziqing_utf8.txt")
docs = loader.load()

# Step 2: Split Document into Chunks (returns List[Document])
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=24)
documents = text_splitter.split_documents(docs)  # This is already a list of Document objects

# Step 3: Load LLMGraphTransformer
#llm = OllamaFunctions(model="llama3.1", temperature=0, format="json")
#llm_graph_transformer = LLMGraphTransformer.from_llm("meta-llama/Llama-3-8B", device="mps")  # Use Apple GPU if available
llm=ChatOpenAI(temperature=0, model_name="gpt-4o")
llm_graph_transformer = LLMGraphTransformer(llm=llm)

# Step 4: Convert the documents to graph documents
graph_documents = llm_graph_transformer.convert_to_graph_documents(documents)

# Step 5: Add the documents to the graph
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [101]:
graph_documents[0]

GraphDocument(nodes=[Node(id='朱自清', type='Person', properties={}), Node(id='荷塘', type='Place', properties={}), Node(id='妻', type='Person', properties={}), Node(id='闰儿', type='Person', properties={}), Node(id='月亮', type='Object', properties={}), Node(id='杨柳', type='Object', properties={}), Node(id='树', type='Object', properties={}), Node(id='叶子', type='Object', properties={}), Node(id='花', type='Object', properties={}), Node(id='月光', type='Object', properties={})], relationships=[Relationship(source=Node(id='朱自清', type='Person', properties={}), target=Node(id='荷塘', type='Place', properties={}), type='THINK_ABOUT', properties={}), Relationship(source=Node(id='朱自清', type='Person', properties={}), target=Node(id='妻', type='Person', properties={}), type='HAS_RELATIONSHIP', properties={}), Relationship(source=Node(id='妻', type='Person', properties={}), target=Node(id='闰儿', type='Person', properties={}), type='HAS_RELATIONSHIP', properties={}), Relationship(source=Node(id='月亮', type='Object',

In [93]:
from neo4j import GraphDatabase

# Load secrets
import os
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")  # Default user unless changed
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# print(f"NEO4J_URI: {NEO4J_URI}")
# print(f"NEO4J_USER: {NEO4J_USER}")
# print(f"NEO4J_PASSWORD: {NEO4J_PASSWORD}")

# Connect to Neo4j
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [ ]:
#embeddings = OllamaEmbeddings(
#    model="mxbai-embed-large",
#)

#dimension=384
#embeddings = OllamaEmbeddings(model="mxbai-embed-base")
#embeddings = OllamaEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")
#embeddings = OllamaEmbeddings(model="all-minilm:l6-v2")
#!pip install -U sentence-transformers

#from langchain.embeddings import OpenAIEmbeddings

#embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key="your_openai_api_key")
#embeddings = OpenAIEmbeddings(model="all-minilm:l6-v2", openai_api_key=os.environ.get("OPENAI_API_KEY"))
#embeddings = OllamaEmbeddings(model="all-minilm:l6-v2", base_url=os.environ.get("OLLAMA_BASE_URL"))
#embeddings = OpenAIEmbeddings(model="all-MiniLM-L6-v2", openai_api_key=os.environ.get("OPENAI_API_KEY"))
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=os.environ.get("OPENAI_API_KEY"))
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

print(f"type of vector_index = {type(vector_index)}, type of vector_retriever = type(vector_retriever)")

: 

In [102]:
def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id`
    FOR (n:__Entity__)
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
# try:
#     create_index()
# except:
#     pass

# Close the driver connection
driver.close()

In [ ]:
class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

# prompt = """请使用中文回答以下问题：
# 问题: {question}
# 答案:
# """

entity_chain = llm.with_structured_output(Entities)

In [115]:
entity_chain.invoke("文章作者是谁?")

Entities(names=['文章作者'])

In [116]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [117]:
print(graph_retriever("文章作者是谁??"))

In [118]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

In [119]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [120]:
chain.invoke(input="文章作者是谁?")

'文章作者是朱自清。'

In [124]:
# entity_chain.invoke("文章是哪一年写的?")
# graph_retriever("文章是哪一年写的?")
chain.invoke(input="文章是哪一年写的? 请使用中文回答")

'文章写于1927年。'

In [111]:
chain.invoke(input="作者怎样来到荷塘的？")

'作者悄悄地披上大衫，带上门，沿着一条曲折的小煤屑路走到了荷塘。'